In [1]:
import pandas as pd
import numpy as np

from tkinter import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

Mengimpor dataset

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/revanfz/revanfz/main/dataset_lirik.csv", encoding="utf-8")

Preprocessing dataset

```
Casefolding : mengubah setiap karakter pada lirik menjadi lowercase (huruf non-kapital)
```

In [3]:
df["lirik"] = df["lirik"].replace(r'\n', ' ', regex=True).str.casefold()

In [ ]:
df

Menghilangkan Stopwords
```
Stopwords adalah kata yang tidak memiliki makna ketika berdiri sendiri
```

In [5]:
id_stop_factory = stopwords.words('indonesian')
en_stop_factory = stopwords.words('english')

Fungsi untuk Menghilangkan stopword bahasa indonesia

In [6]:
def remove_stop_word_id(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in id_stop_factory:
            clean_words.append(word)
    return ' '.join(clean_words)


Fungsi untuk Menghilangkan stopwords bahasa inggris

In [7]:
def remove_stop_word_en(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in en_stop_factory:
            clean_words.append(word)
    return ' '.join(clean_words)


Menghilangkan stopwords pada lirik

In [8]:
df["lirik"].apply(remove_stop_word_id)
df["lirik"].apply(remove_stop_word_en)

0     la, la, la, la-la-la la, la-la-la, la, la-la-l...
1     entah sudah selasa yang ke berapa masih saja k...
2     kulepas semua yang kuinginkan tak akan kuulang...
3     there's way time may find unknown then, loner ...
4     georgia wrap your— want ya arms, oh, let hold ...
5     kalau harus ku mengingatmu lagi aku takkan san...
6     eighteen, undergrads stayed late, never made c...
7     sandarkan lelahmu, dan ceritakan tentang apa p...
8     kau begitu sempurna di mataku kau begitu indah...
9     lihatlah luka ini yang sakitnya abadi yang ter...
10    aku sedang bertanya-tanya tentang perasaan kit...
11    saat kau jatuh lukai hati di mana pun itu i'll...
12    malam ini aku menanti kedatanganmu mengisi sep...
13    know want secret try hide know want don’t keep...
14    wants beautiful goes unnoticed, knows limits c...
Name: lirik, dtype: object

Vektorisasi lirik
```
Vektorisasi adalah proses untuk mengubah proses mengubah setiap term menjadi bentuk vektor 
yang akan digunakan untuk digunakan sebagai fitting dalam prediksi 
machine learning

Fitting adalah proses yang bertujuan membuat model beradaptasi terhadap dataset
yang ia pakai untuk training
```

In [9]:
tfidf_v = TfidfVectorizer()
vektor_lirik = tfidf_v.fit_transform(df["lirik"].tolist())

Fungsi untuk mencari lagu
yang meliputi proses:
```
1. Preprocessing query (casefolding, removing stopwords)
2. Vektorisasi query
3. Menghitung nilai cosine similarity
4. Mengurut nilai cosine similarity secara descending (besar -> kecil)
5. Menampilkan GUI window yang menampilkan hasil pencarian
```

In [10]:
def cari_lagu(input_text, window):
    query = input_text.get().lower().split(' ')
    clean_query = ""
    for i in query:
        if i not in id_stop_factory and i not in en_stop_factory:
            clean_query += i + " "
        else:
            pass

    clean_query = clean_query.strip()
        
    query_vektor = tfidf_v.transform([clean_query])
    cos_sim = cosine_similarity(query_vektor, vektor_lirik)
    sorted_index = np.argsort(cos_sim)[0][::-1]
    
    global index_relevan
    index_relevan = []
    for i in sorted_index:
    # Memfilter hasil dokumen yang memiliki nilai cosine similarity > 0
        if cos_sim[0][i] > 0:
            index_relevan.append([i, cos_sim[0][i]])
            
    for i in index_relevan:
        print("Dokumen {} memiliki cosine similarity sebesar {:.2f}".format(i[0], i[1]))
        
    print(f"Dokumen relevan yang dihasilkan sistem yakni : {len(index_relevan)} ")

    global lagu
    lagu = []
    for i in index_relevan:
        x = df.iloc[i[0]]['judul']
        y = df.iloc[i[0]]['artis']
        z = df.iloc[i[0]]['lirik']
        lagu.append([x, y, z])

    if len(lagu) > 0:
        target = lagu[0][0]
    else:
        target = "Lagu tidak ditemukan."

    hasil = Frame(window)
    hasil.grid(row=0, column=0, sticky='news')
    hasil.tkraise()
    Label(hasil, text="Lagu yang anda cari adalah: ",  font=("Times",20,"bold")).pack()
    
    count = 1
    for i in lagu:
        if count == 2:
            Label(hasil, text="atau mungkin: ").pack()
        Label(hasil, text=f"{count}.\t\t{i[0]}").pack()
        Label(hasil, text=f"oleh {i[1]}").pack()
        count += 1

    Button(hasil, text="search again", command=lambda:hasil.destroy()).pack()
    Button(hasil, text="close", command=lambda:window.destroy()).pack()

Membuat GUI Window menggunakan library tkinter

In [20]:
window = Tk()
input_text = StringVar()
window.geometry("480x480")
window.title("Song Finder")

homepage = Frame(window)
homepage.grid(row=0, column=0, sticky='news')
Label(homepage, text="Song Finder", bg="teal", fg="white").pack()
Entry(homepage, text="Masukkan penggalan lirik", textvariable=input_text).pack()
Button(homepage, text="Cari", command=lambda:cari_lagu(input_text, window)).pack()

homepage.tkraise()
window.mainloop()

Dokumen 4 memiliki cosine similarity sebesar 0.22
Dokumen 0 memiliki cosine similarity sebesar 0.01
Dokumen 13 memiliki cosine similarity sebesar 0.01
Dokumen 14 memiliki cosine similarity sebesar 0.01
Dokumen relevan yang dihasilkan sistem yakni : 4 


Pendefinisian dokumen relevan (menurut ahli)

In [23]:
dokumen_relevan_sebenarnya = {
    'kau tetap': [1, 7, 9], 
    'all the time': [0],
    'lost within the darkness': [4],
    "we're strolling down the boulevard": [6],
    'kau adalah hidupku' : [8]
}

Perhitungan evaluasi sistem terhadap kueri yang dimasukkan

In [13]:
avg_precision_list = {}

In [21]:
dokumen_relevan_sebenarnya_yang_dihasilkan = 0
dokumen_ke = 0
precision = 0
avg_precision = 0
sum_precision = 0

for i in index_relevan:
    dokumen_ke += 1
    if i[0] in dokumen_relevan_sebenarnya[input_text.get()]:
        dokumen_relevan_sebenarnya_yang_dihasilkan += 1
        sum_precision += dokumen_relevan_sebenarnya_yang_dihasilkan/dokumen_ke

if len(dokumen_relevan_sebenarnya[input_text.get()]) > 0:
    recall = dokumen_relevan_sebenarnya_yang_dihasilkan/len(dokumen_relevan_sebenarnya[input_text.get()])
    precision = dokumen_relevan_sebenarnya_yang_dihasilkan/len(index_relevan)
else:
    recall = 1
    precision = 1

avg_precision = sum_precision/len(dokumen_relevan_sebenarnya[input_text.get()])
avg_precision_list[input_text.get()] =  avg_precision

print(f"Kueri \"{input_text.get()}\" memiliki nilai:")
print(f"Recall: {int(recall*100)} %")
print(f"Precision: {int(precision*100)} %")
print(f"dan Average Precision: {int(avg_precision*100)} %")

Kueri "lost within the darkness" memiliki nilai:
Recall: 100 %
Precision: 25 %
dan Average Precision: 100 %


In [22]:
mAP = sum(avg_precision_list.values())/len(avg_precision_list)
print(f"Nilai dari Mean Avrage Precision sistem adalah : {mAP}")

Nilai dari Mean Avrage Precision sistem adalah : 0.7777777777777777
